In [2]:
# Import dependencies
import pandas as pd
from path import Path 
from collections import Counter

# Read data from csv into new dataframe
data = 'cc_default.csv'
df = pd.read_csv(data)
df.head()


,ID,ln_balance_limit,sex,education,marriage,age,default_next_month
0,1,9.903488,1,2,0,24,1
1,2,11.695247,1,2,1,26,1
2,3,11.407565,1,2,1,34,0
3,4,10.819778,1,2,0,37,0
4,5,10.819778,0,2,0,57,0


In [3]:
# Define features and target
X = df.drop(columns=['ID', 'default_next_month'])
y = df['default_next_month']

In [4]:
# Split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [5]:
# Use RandomUnderSampler to train the instance, then undersample the majority class (the one with more data points)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 4968, 1: 4968})

In [6]:
# Again, we can now use a Logistic Regression on the dataset
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [7]:
# Make predictions and generate confusion matrix of the results
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[3732, 2100],
       [ 740,  928]], dtype=int64)

In [8]:
## Calculate balanced accuracy score
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.64      0.56      0.72      0.60      0.36      5832
          1       0.31      0.56      0.64      0.40      0.60      0.35      1668

avg / total       0.72      0.62      0.57      0.65      0.60      0.36      7500



# Results are not too impressive at predicting defaults based on given features. Next we will try Cluster Centroid Undersampling to see if our results improve.

In [9]:
# We can utilize the same X and y train data sets we created previously

# Instantiate the resampling module and resample the data using Cluster Centroids
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [11]:
# Instantiate and train logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Generate confusion matrix, balanced accuracy score, and classification report as we did for Random Under Sampling above
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.49      0.63      0.61      0.56      0.31      5832
          1       0.26      0.63      0.49      0.37      0.56      0.31      1668

avg / total       0.70      0.52      0.60      0.56      0.56      0.31      7500



# Results are worse than using the random undersampling method, which shows that even a more complex resampling method does not necessarily lead to better results.